https://pypi.org/project/requests-oauth2client/ -- Check for oaut2client documentation


In [105]:
from requests_oauth2client import *
import credentials as c
import json
import math
import pandas as pd

token_endpoint = c.token_endpoint
client_id = c.client_id
client_secret = c.client_secret
api_url = c.api_url
scope = c.scope

oauth2client = OAuth2Client(
    token_endpoint=token_endpoint,
    auth=(client_id, client_secret),
)

auth = OAuth2ClientCredentialsAuth(
    oauth2client, scope=scope, resource=api_url
)

api = ApiClient(
    api_url, auth=auth
)

In [106]:
resp_cl = api.get("/clients")

# Decode the response content and parse the JSON data
response_cl_data = resp_cl.content.decode('utf-8')

cl_data = json.loads(response_cl_data)

total_count = cl_data['data']['info']['count']
per_page = cl_data['data']['info']['per_page']
current_page = cl_data['data']['info']['page']

num_pages = math.ceil(total_count / per_page)


# List to store all the client records
clients= []

for page in range(1, num_pages + 1):
    resp_cl = api.get(f"/clients?page={page}")
    response_cl_data = resp_cl.content.decode('utf-8')
    page_data = json.loads(response_cl_data)
    if page_data['data'] is not None:
        #clients_records.update(page_data['data'])
        for client in page_data['data']['clients']:
            if client['group'] is not None:
                group_name = client['group']['name']
            else:
                group_name = None

            client_info = [
                client['number'],
                client['email'],
                client['name'],
                client['surname'],
                client['phone'],
                client['company'],
                client['nip'],
                group_name,
                client['additional']['1'],
                client['additional']['2'],
                client['additional']['3']
            ]
            clients.append(client_info)

# Creating customer id which are used inside of org
for client in clients:
    client[0] = 'BB_' + client[0]

In [112]:
columns = ['client_id', 'client_email', 'client_name', 'client_surname', 'client_phone', 'company', 'client_nip', 'account_name', 'salesrep', 'products', 'delivery']
df = pd.DataFrame(clients)
df.columns = columns
df.to_excel('output/sellout_clients.xlsx', index=False)